# LangGraph Neo4j MCP Agent (Simple)

A simplified notebook for querying a Neo4j database using LangGraph and the Model Context Protocol (MCP).

This version uses a static access token - no automatic token refresh.

## 1. Install Dependencies

In [ ]:
%pip install --upgrade --quiet \
    langchain>=0.3.14 \
    langgraph>=0.2.60 \
    langchain-aws>=0.2.10 \
    langchain-mcp-adapters>=0.2.1 \
    mcp>=1.3.0 \
    httpx>=0.28.0 \
    boto3>=1.36.0 \
    nest-asyncio>=1.6.0

print("Done! Restart the kernel below.")

## 2. Restart Kernel

Run this cell after installing packages, then continue from Section 3.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 3. Imports

In [ ]:
import asyncio
import nest_asyncio
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_mcp_adapters.client import MultiServerMCPClient

nest_asyncio.apply()
print("Imports complete.")

## 4. Configuration

### **ACTION REQUIRED**

Open your `.mcp-credentials.json` file and copy the values into the cell below:

```json
{
  "gateway_url": "<-- copy this to GATEWAY_URL",
  "access_token": "<-- copy this to ACCESS_TOKEN",
  ...
}
```

> **Note**: Tokens typically expire after 1 hour. If you get authentication errors, get a fresh token.

In [ ]:
# =============================================================================
# REPLACE THESE VALUES with your credentials from .mcp-credentials.json
# =============================================================================

GATEWAY_URL = "YOUR_GATEWAY_URL_HERE"  # e.g., "https://xxx.execute-api.us-west-2.amazonaws.com/mcp"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN_HERE"  # e.g., "eyJhbGciOiJSUzI1NiIsInR5cCI6..."

# =============================================================================
# AWS Bedrock settings (usually no changes needed)
# =============================================================================

AWS_REGION = "us-west-2"
MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"

# Validate configuration
if "YOUR_" in GATEWAY_URL or "YOUR_" in ACCESS_TOKEN:
    print("ERROR: Please replace GATEWAY_URL and ACCESS_TOKEN with your actual credentials!")
    print("       Open .mcp-credentials.json and copy the values.")
else:
    print(f"Gateway: {GATEWAY_URL[:50]}...")
    print(f"Token: {ACCESS_TOKEN[:20]}...")
    print(f"Region: {AWS_REGION}")
    print("Configuration OK!")

## 5. System Prompt

In [ ]:
SYSTEM_PROMPT = """You are a helpful Neo4j database assistant with access to tools that let you query a Neo4j graph database.

Your capabilities include:
- Retrieve the database schema to understand node labels, relationship types, and properties
- Execute read-only Cypher queries to answer questions about the data
- Do not execute any write Cypher queries

When answering questions about the database:
1. First retrieve the schema to understand the database structure
2. Formulate appropriate Cypher queries based on the actual schema
3. If a query returns no results, explain what you looked for and suggest alternatives
4. Format results in a clear, human-readable way
5. Cite the actual data returned in your response

Important Cypher notes:
- Use MATCH patterns that align with the actual schema
- For counting, use MATCH (n:Label) RETURN count(n)
- For listing items, add LIMIT to avoid overwhelming results
- Handle potential NULL values gracefully

Be concise but thorough in your responses."""

## 6. Create Agent

In [ ]:
async def setup_agent():
    """Create the MCP client and LangGraph agent."""
    print("Connecting to MCP server...")
    
    client = MultiServerMCPClient(
        {
            "neo4j": {
                "transport": "streamable_http",
                "url": GATEWAY_URL,
                "headers": {
                    "Authorization": f"Bearer {ACCESS_TOKEN}",
                },
            }
        }
    )
    
    tools = await client.get_tools()
    print(f"Loaded {len(tools)} tools:")
    for tool in tools:
        print(f"  - {tool.name}")
    
    print(f"\nInitializing LLM ({MODEL_ID})...")
    llm = init_chat_model(
        MODEL_ID,
        model_provider="bedrock_converse",
        region_name=AWS_REGION,
        temperature=0,
    )
    
    print("Creating agent...")
    agent = create_agent(llm, tools, system_prompt=SYSTEM_PROMPT)
    print("Agent ready!")
    
    return agent

# Create the agent
agent = asyncio.get_event_loop().run_until_complete(setup_agent())

## 7. Query Helper

In [ ]:
async def ask(question: str) -> str:
    """Ask the agent a question about the Neo4j database."""
    print("=" * 70)
    print(f"Q: {question}")
    print("=" * 70)
    
    result = await agent.ainvoke({"messages": [("human", question)]})
    
    messages = result.get("messages", [])
    if messages:
        content = getattr(messages[-1], "content", str(messages[-1]))
        print(f"\nA:\n{content}")
        return content
    return "No response"

def query(question: str) -> str:
    """Synchronous wrapper for ask()."""
    return asyncio.get_event_loop().run_until_complete(ask(question))

## 8. Demo Queries

In [ ]:
# Get database schema
_ = query("What is the database schema? Give me a brief summary.")

In [ ]:
# Count nodes
_ = query("How many nodes are in the database by label?")

In [ ]:
# Explore relationships
_ = query("What types of relationships exist in the database?")

## 9. Your Queries

Replace the question and run the cell.

In [ ]:
_ = query("List 5 sample records from the most populated node type.")

In [ ]:
# Add more queries here
# _ = query("Your question here")